This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("data/mra.mha",itk.F)

In [3]:
res = ttk.ResampleImage[ImageType].New()
res.SetInput(im1)
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

In [4]:
xp1=[-3.9,-54.8,12.3]
xp2=[18.4,-16,-21]
xp3=[-46,-15,-13]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(False)
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetVerbose(True)
tube = vSeg.ExtractTubeInObjectSpace( xp1, 1 )
tube = vSeg.ExtractTubeInObjectSpace( xp2, 2 )
tube = vSeg.ExtractTubeInObjectSpace( xp3, 3 )
tubeG = vSeg.GetTubeGroup()

In [10]:
# Example of accessing data in a tube
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
r = list(map(lambda x: tube.GetPoint(x).GetNormal1InObjectSpace()[0], range(tube.GetNumberOfPoints())))
print(r)

[0.78819174722339, -0.11883820489835029, -0.2408084936194763, -0.2006114429408959, 0.2837980654727114, -0.25958842305894697, -0.95634799500795, -0.19942995619527898, -0.20482151726584308, -0.02473260220704085, -0.07450891067254098, 0.023165082730034037, 0.889696706827139, 0.8842970055599659, 0.8778867607040712, 0.8718426401749201, 0.8660263771089827, 0.856445348550502, 0.7553988682568373, -0.13305221465386705, 0.1638135472442974, 0.4535108502837055, 0.7754186822989052, 0.738207073717042, 0.745669689433066, 0.7235777216391824, 0.35459730985310384, -0.1437106695471884, 0.5694218657018256, 0.2699973667841763, 0.6242262208340412, -0.3642428673165844, 0.4553090471566555, 0.4175666326068665, 0.7642031551196297, 0.7636264736430833, 0.7629957498979016, 0.7623007445339436, 0.8561974631400677, 0.7509248682142828, -0.8321744183866263, 0.7575815364693009, -0.19203068235977694, 0.45563533192137273, 0.9065076652703595, 0.450805780470459, -0.8034899236989779, 0.749990867665099, 0.7481929263283558, 0.

In [6]:
TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetColorByRidgeness(True)
tubesToImageFilter.SetTemplateImage(im1)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()
view(outputImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [13]:
writer = itk.SpatialObjectWriter[3].New()
writer.SetFileName("vessels.tre")
writer.SetInput(tubeG)
writer.Update()